In [1]:
import pandas as pd
from collections import Counter
from scipy.sparse import csr_matrix
import math
from tqdm import tqdm
from sklearn.preprocessing import normalize
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC



In [2]:
# df = pd.read_csv('/content/drive/MyDrive/ML DATA/finalized_data.csv')
df = pd.read_csv('data_for_training.csv')

In [3]:
class TFIDFVectorizer:
    def __init__(self):
        self.vocab = {}
        self.idfs = {}

    def _idf(self, corpus, unique_words):
        idf_vals = {}
        total_docs = len(corpus)
        for word in tqdm(unique_words):
            cnt = 0
            for row in corpus:
                if word in row.split(" "):
                    cnt += 1
            idf_vals[word] = 1 + math.log((1 + total_docs) / (1 + cnt))
        return idf_vals

    def fit(self, dataset):
        if isinstance(dataset, (list,)):
            unique_words = set()
            for row in tqdm(dataset):
                for word in row.split(" "):
                    if len(word) < 2:
                        continue
                    unique_words.add(word)
            unique_words = sorted(list(unique_words))
            self.vocab = {j: i for i, j in enumerate(unique_words)}
            self.idfs = self._idf(dataset, unique_words)
        return self

    def transform(self, dataset):
        rows, cols, data = [], [], []
        for idx, row in tqdm(enumerate(dataset), total = len(dataset)):
            word_count = Counter(row.split(' '))
            for word, count in word_count.items():
                if word in self.vocab:
                    tf = count / len(row.split(' '))
                    tfidf = tf * self.idfs[word]
                    rows.append(idx)
                    cols.append(self.vocab[word])
                    data.append(tfidf)
        sparse_matrix = csr_matrix((data, (rows, cols)), shape=(len(dataset), len(self.vocab)))
        return normalize(sparse_matrix, norm='l2', axis=1, copy=True)

    def fit_transform(self, dataset):
        self.fit(dataset)
        return self.transform(dataset)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12231 entries, 0 to 12230
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  12231 non-null  object
 1   Text      12231 non-null  object
dtypes: object(2)
memory usage: 191.2+ KB


# სამწუხაროდ ჩემი დაწერილი TF-IDF დიდ კორპუსზე ძალიან ნელა მუშაობს, ამიტომ მიწევს sklearn-ის გამოყენება :(

In [5]:
tfidf = TfidfVectorizer( norm=None)
tfidf.fit(df["Text"])
weights = tfidf.transform(df["Text"])


In [6]:
le = LabelEncoder()
le.fit(df["Category"])
df["Category"] = le.transform(df["Category"])

In [7]:
df["Category"].unique()


array([ 0,  1,  2,  3,  4,  5,  7,  8, 10, 11,  9, 12, 13, 14, 15, 16, 17,
       18, 19, 21, 22, 24, 25, 23, 26, 27, 30, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 45, 43, 44, 46, 47, 28, 31,  6, 20, 29])

ახლა, ამ სეტაპით მინდა განვიხილო რამდენიმე მოდელი:

1.   **KNN**

2.   **logistic regression**

3.   **SVM**

4.   **Random Forest Regression**



In [8]:
weights[0].shape

(1, 118697)

ერთ-ერთი რამ, რაც უნდა გვექნა აქამდე და არ ვქენით, არის weight-ების dimension-ის ცვლილება. ერთ-ერთი ამის ვარიანტი არის SVD ფაქტორიზაცია.

In [9]:
svd = TruncatedSVD(n_components=1200, random_state=42)
reduced_weights = svd.fit_transform(weights)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(reduced_weights, df["Category"], test_size=0.1, random_state=42)

In [11]:
models = {
    'KNN': KNeighborsClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=50),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC()

}
accuracy_scores = {}
# Train and evaluate each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores[model_name] = accuracy

    # Print the accuracy
    print(f'Accuracy of {model_name} on test set: {accuracy:.2f}')


C:\anacondaPY3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


Accuracy of KNN on test set: 0.68


C:\anacondaPY3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy of LogisticRegression on test set: 0.78
Accuracy of RandomForest on test set: 0.73
Accuracy of SVM on test set: 0.75


ეს სივი არის დაგენერირებული ჯიპიტის მიერ. არანაირი ინსტრუქცია, არანაირი დატის წმენდა არ ჩამიტარებია, არაფერი არ მითქვამს გარდა იმისა, რომ ფიტნეს ინსტრუქტორის სივი დაეგენერირებინა.

In [12]:
cv_text = """John Doe
123 Fitness Ave
Gym City, GA 30303
(555) 123-4567
johndoe@example.com
LinkedIn: linkedin.com/in/johndoe-fitness
Objective
Dynamic and certified Health and Fitness Instructor with over 5 years of experience specializing in creating personalized workout programs and conducting high-energy group fitness classes. Dedicated to helping clients achieve their health and fitness goals through tailored exercise routines and comprehensive nutritional guidance.
Certifications
Certified Personal Trainer, American Council on Exercise (ACE), 2018
Group Fitness Instructor, National Academy of Sports Medicine (NASM), 2019
CPR and First Aid, American Red Cross, 2020
Yoga Instructor, 200-Hour RYT, Yoga Alliance, 2017
Professional Experience
Health and Fitness Instructor
Gold's Gym, Atlanta, GA
June 2020 – Present
Design and implement customized fitness programs for over 100 regular clients, increasing their fitness performance by an average of 25%.
Lead weekly group fitness classes, including Spin, HIIT, and Yoga, with an average attendance of 30 participants per class.
Conduct bi-weekly workshops on nutrition and wellness that have improved client retention by 20%.
Utilize motivational interviewing techniques to encourage client commitment and achieve a 95% success rate in client goal attainment.
Fitness Coach
Anytime Fitness, Atlanta, GA
May 2015 – May 2020
Spearheaded a successful bootcamp program that grew to include 50 regular participants.
Collaborated with physical therapists to design rehabilitation exercises that safely engaged clients in physical activity post-injury.
Increased gym membership by organizing monthly health and wellness fairs that attracted over 200 attendees.
Education
Bachelor of Science in Exercise Science
Georgia State University, Atlanta, GA
August 2011 – May 2015
Skills
Expert in creating tailored fitness programs.
Proficient with digital fitness tracking systems like Fitbit and MyFitnessPal.
Excellent communicator with effective client-facing skills.
Knowledgeable in body mechanics and functional training.
Fluent in Spanish.
Professional Affiliations
Member, National Strength and Conditioning Association (NSCA)
Volunteer, Community Fitness Days, providing free fitness coaching to underprivileged communities
References
Available upon request."""

# Assuming 'vectorizer' is already fitted to a relevant corpus
tfidf_weights = tfidf.transform([cv_text])
reduced_weights = svd.transform(tfidf_weights)

In [13]:
reduced_weights.shape

(1, 1200)

In [14]:
for model_name, model in models.items():
  label = model.predict(reduced_weights)
  decoded_label = le.inverse_transform([label[0]])
  print(f"Decoded label for {model_name}:", decoded_label)

Decoded label for KNN: ['Health and Fitness']
Decoded label for LogisticRegression: ['Health and Fitness']
Decoded label for RandomForest: ['Health and Fitness']
Decoded label for SVM: ['Health and Fitness']


C:\anacondaPY3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


შემდეგ, გავტესტავ ჩემზე და ჩემი მეგობრების სივი-ებზე.

In [15]:
def prepare_weights(text):
    tfidf_weights = tfidf.transform([text])
    reduced_weights = svd.transform(tfidf_weights)
    return reduced_weights
def print_model_outputs(weights):
    for model_name, model in models.items():
      label = model.predict(weights)
      decoded_label = le.inverse_transform(label)
      print(f"Decoded label for {model_name}:", decoded_label)

In [16]:
from preprocess.process import prepare
path = "C:/Users/User/Desktop/Nika Mikaberidze.pdf"
def output_everything(path):
    text = prepare(path)
    weights = prepare_weights(text)
    print_model_outputs(weights)
    return text

In [17]:
txt1 = output_everything(path)

Decoded label for KNN: ['Data Science']
Decoded label for LogisticRegression: ['Data Science']
Decoded label for RandomForest: ['Data Science']
Decoded label for SVM: ['Data Science']


C:\anacondaPY3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [23]:
print(txt1)

Nika Mikaberidze
TbilisiGeorgia   10 97 97  
EDUCATION
Free University Of Tbilisi
Tbilisi Georgia
Computer And Electrical Engineering
Graduation Date Sep 
WORK EXPERIENCE
Supernova AI
Tbilisi Georgia
Machine Learning Intern NLP

Mar 
Jun 
Gained valuable exposure to the dynamic field of Data Science and Natural Language Processing broadening my 
technical expertise and analytical skills
Utilized a wide range of data analysis libraries for indepth conversational AI research and contributed to the creation 
of a comprehensive EnglishGeorgian semantic search database at Supernova AI
AI Lab
Tbilisi Georgia
Junior Data Scientist

Jul 
Present
Engaged in extensive AWS toolset applications utilizing SageMaker for iterative model training and S3 for efficient 
data handling and storage solutions
Played a key role in localizing Microsofts TinyStories research adding a Georgian translation layer and adapting 
27M of English data to Georgian ensuring domainspecific relevance and surpassing Google

In [19]:
# ეს ჩემი მეგობრის სივია. ერთ-ერთ პუნქტში AI-ზე მუშაობდა, ერთ-ერთში wildfire monitoring-ზე და ერთ-ერთში დატის შეგროვების ამოცანაზე, საბოლოოდ ელ. ინჟინერია, მაგრამ ფაქტია, მოდელი ამას ვერ მიხვდა. თუმცა, მოდელი კი არა, მეც ვერ მივხვდებოდი...
path_2 = "C:/Users/User/Downloads/CV.pdf"
txt2 = output_everything(path_2)

Decoded label for KNN: ['Data Science']
Decoded label for LogisticRegression: ['Information Technology']
Decoded label for RandomForest: ['Data Science']
Decoded label for SVM: ['Data Science']


C:\anacondaPY3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [24]:
print(txt2)

Mariam Rekhviashvili
Tbilisi Georgia       Linkedin
EDUCATION
Free University of Tbilisi
Tbilisi
Bachelor of Engineering Electric and Computer Engineering
Graduation Date Sep 
PROJECT EXPERIENCE
GITA Nasa Space Apps Challenge 
Tbilisi
Wildfire Monitoring System

Oct 
Oct 
Designed and engineered a hardware solution equipped with multiple sensors including infrared radiation carbon 
monoxide temperature and humidity sensors to accurately detect and assess wildfire risks in specific environments 
contributing to early detection and prevention efforts
Collaborated with a multidisciplinary team to conceptualize and deploy a sophisticated algorithm that analyzed data 
from the sensor array to calculate the probability of wildfire occurrences empowering local communities and 
monitoring groups with actionable insights for proactive fire management and resource allocation
GITA Citython 
Kutaisi
Transit Tracer

Oct 
Oct 
Designed a robust hardware infrastructure incorporating laser sensors to 

In [21]:
# ეს ჩვენი მეგობრის და გუნდელის, ლუკა დარსალიას სივია.
path_3 = "C:/Users/User/Downloads/ML-Resume-2.pdf"
txt3 = output_everything(path_3)

Decoded label for KNN: ['Web Designing']
Decoded label for LogisticRegression: ['React Developer']
Decoded label for RandomForest: ['Data Science']
Decoded label for SVM: ['Data Science']


C:\anacondaPY3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [25]:
print(txt3)

Luka Darsalia
   70 06 28 Georgia Tbilisi GitHub 
Linkedin
WORK EXPERIENCE
AI Lab
Aug   Present
ML Engineer
Georgia Tbilisi

Developed a Georgian Small Language Model SLM with 161M parameters inspired by Microsofts Tiny
Stories research achieving superior performance in story generation and QA tasks

Engineered a Neural Machine Translation NMT system outperforming Google Translate and other services in
accuracy for English to Georgian translations crucial for highquality content creation and localization

Pioneered a custom instruction dataset posttranslation enhancing the models conversational abilities and
enabling the generation of nuanced culturally relevant tiny stories from prompts
Supernovaai
Mar   Jun 
ML Engineer Intern
Georgia Tbilisi

Developed and implemented a comprehensive data preprocessing pipeline

Created a highly accurate Georgian language model for semantic search on an online shopping platform

Successfully implemented advanced searching algorithms including Fuzzy 

# საბოლოოდ, ვფიქრობ უკეთესი/სხვა მიდგომებიც არსებობს, ვიდრე უბრალოდ სტატისტიკურად TF-IDF-ით გაკეთება. ამას, შემდეგ ნოუთბუქში ვნახავთ.

In [26]:
for i, item in enumerate([txt1, txt2, txt3]):
    with open(f"CV{i}.txt", "w") as f:
        f.write(item)